# ConTextTab test

In [88]:
import numpy as np
import pandas as pd
import os
import torch
import transformers
from huggingface_hub import login
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score

In [89]:
from sap_rpt_oss import SAP_RPT_OSS_Classifier

In [90]:
kaisaniemi = pd.read_csv("sample_data/Kaisaniemi_2023-2025.csv")
kumpula = pd.read_csv("sample_data/Kumpula_2025.csv")
df = kaisaniemi
#df = kumpula
df.shape

(1096, 10)

In [91]:
#df = df[df["Vallitseva sää"].isin(['Ei merkittäviä sääilmiöitä ', 'Heikkoa lumisadetta', 'Heikkoja vesikuuroja', 'Utua', 'Heikkoa vesisadetta'])]
df = df[df["Vallitseva sää"].isin(['Heikkoa lumisadetta', 'Heikkoja vesikuuroja', 'Utua'])]
df.shape

(114, 10)

In [92]:
X = df.drop("Vallitseva sää", axis=1)
y = df["Vallitseva sää"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

classifier = SAP_RPT_OSS_Classifier(max_context_size=2048, bagging=1)

classifier_contextab = classifier.fit(X_train, y_train)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [93]:
y_predict = classifier_contextab.predict(X_test)
acc = accuracy_score(y_test, y_predict)
y_predict = pd.Series(y_predict)
print(f"accuracy: {acc}")
results = pd.concat([y_test.reset_index(drop=True), y_predict.reset_index(drop=True)], axis=1)
results.columns = ["correct_label", "prediction"]
results.head(10)

Data type str not recognized! Defaulting to string
Data type str not recognized! Defaulting to string
Data type str not recognized! Defaulting to string
Data type str not recognized! Defaulting to string
Data type str not recognized! Defaulting to string
Data type str not recognized! Defaulting to string
accuracy: 0.782608695652174


,correct_label,prediction
0,Heikkoja vesikuuroja,Heikkoja vesikuuroja
1,Utua,Heikkoa lumisadetta
2,Utua,Utua
3,Heikkoa lumisadetta,Heikkoa lumisadetta
4,Heikkoa lumisadetta,Heikkoa lumisadetta
5,Heikkoa lumisadetta,Heikkoa lumisadetta
6,Heikkoja vesikuuroja,Heikkoja vesikuuroja
7,Heikkoa lumisadetta,Heikkoa lumisadetta
8,Heikkoa lumisadetta,Heikkoa lumisadetta
9,Heikkoja vesikuuroja,Heikkoja vesikuuroja
